In [9]:
import numpy as np

In [10]:
"""
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
https://en.wikipedia.org/wiki/Glossary_of_blackjack_terms
"""

# configurations
NDECKS: int = 8
DECK_PENETRATION: float = 0.25 # typical number of decks dealt out before reshuffle
# NPLAYERS: int = 4 # affects how many cards you may be able to count

In [11]:
"""
We keep basic card face and suit information. 
Later, could implement weird house rules without major modification.
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings to enable Cython memview of C char[2] dtypes
shoe = shoe.astype('S2', order='C')

cut_idx = min(round((len(deck) * NDECKS) * DECK_PENETRATION), len(deck) * NDECKS)

In [12]:
# we need this to be fast, we will shuffle a lot!
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

7.99 µs ± 89.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
# BAD: redundant for 1D array. 
shoe_size = shoe.shape[0]
%timeit shoe[rng.permutation(shoe_size)]

11.7 µs ± 57.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
# BAD idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards.astype('S1'), values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

66.4 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
# may change to use numpy to read-in csv, to avoid pandas dependency
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

"""
Read in the individual strategy tables. Cast into byte chars and C contiguous arrays. 
"""
hard = strat.iloc[0:18, :].copy().reset_index(drop=True)
hard = hard.values.astype(str)
hrd = hard[1:,1:].astype('S1', order='C')

soft = strat.iloc[18:27, :].copy().reset_index(drop=True)
soft = soft.values.astype(str)
sft = soft[1:,1:].astype('S1', order='C')

splits = strat.iloc[27:38, :].copy().reset_index(drop=True)
splits = splits.values.astype(str)
splts = splits[1:,1:].astype('S1', order='C')

In [15]:
# Read in the counting strategy
count = pd.read_csv(r"StrategyInput\HiLoCount.csv", header=None)
cnt = count.iloc[:,1].values.astype(float, order='C')

In [16]:
dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

In [6]:
import CardStream
print(CardStream.some_func(5))

5


In [23]:
%%cython --cplus
# do not know if this global directive is actually needed - maybe later when we break into files
# # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
# cython: c_string_type=str, c_string_encoding=ascii

import numpy as np
cimport cython

cdef int ASCII_A = 65 
cdef unsigned int MEMBLOCKS = 21 # LARGEST POSSIBLE CARD SIZE OF HAND (21 SOFT ACES)

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# Python dicts actually implemented quite fast
# If this is too slow, can consider using unordered_map from C++ STL. 
# https://stackoverflow.com/questions/29268914/python-dictionaries-vs-c-stdunordered-map-cython-vs-cythonized-python-dict
# https://stackoverflow.com/questions/32266444/using-a-dictionary-in-cython-especially-inside-nogil
cdef dict dealer_idx = {
    b'2': 0,
    b'3': 1,
    b'4': 2,
    b'5': 3,
    b'6': 4,
    b'7': 5,
    b'8': 6,
    b'9': 7,
    b'T': 8,
    b'J': 8,
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

ctypedef unsigned int[MEMBLOCKS] hand_t

# function decorators to turn off various things, in the interest of speed
# https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(
    card_dtype_struct[:] shoe not None, 
    Py_ssize_t cut_idx, 
    char[:,:] hrd not None, 
    char[:,:] sft not None, 
    char[:,:] splt not None, 
    double[:] cnt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 

    Inputs: (REFER TO TEMPLATE FILES IN STRATEGY INPUT)
        - 'shoe': numpy array (buffer) of char[2] byte strings
        - 'cut_idx': int, simulates the depth of the cut card
        - 'hrd': numpy array (buffer) of char bytes, in the input format dictating BJ action on player hard hand
        - 'sft': numpy array (buffer) of char bytes, in the input format dictating BJ action on player soft hand
        - 'splt': numpy array (buffer) of char bytes, in the input format dictating BJ action on player paired hand
        - 'cnt': numpy array (buffer) of double (Python float6), in the input format, with count values
    """

    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef double count = 0 # card count

    # declare arrays to hold idxs of cards for player and dealer
    cdef hand_t phand
    cdef hand_t dhand 

    # play the shoe
    while dealt < cut_idx:
    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
        
        # init both our hands to blanks

        break


    print(bool(shoe[0].face == b'J'))

    # type cast into python bytes, so works as dict key
    # https://cython.readthedocs.io/en/latest/src/tutorial/strings.html
    print(dealer_idx.get(<bytes>(shoe[0].face)))





Error compiling Cython file:
------------------------------------------------------------
...
    b'Q': 8,
    b'K': 8,
    b'A': 9,
}

ctypedef unsigned int[MEMBLOCKS] hand_t
                              ^
------------------------------------------------------------

C:\Users\tp\.ipython\cython\_cython_magic_30aacef077600b11a75d49835af6b72a.pyx:37:31: Not allowed in a constant expression
